The project consists in comparing at least two different methods for unsupervised domain adaptation and showing the results in a table. The results shall show performance on the source dataset, and performance on the target dataset. Moreover, the upper bound will be evaluated by training and testing the models on the target dataset as a sort of oracle predictor.


Google Colab was used for the project for computational reasons.

---
## Initialization

Since I'm using Google Colab for computational reasons, I will structure this notebook by putting in these first cells all relevant hyperparameters and libraries.


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#%cd 'insert path'

In [ ]:
import os
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from tqdm import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(torch.cuda.get_device_name(0))
print(device)


Parameters:

In [ ]:
batch_size = 64
num_epochs = 30
learning_rate=0.0001
source_train_dir='AFE\\train'
source_test_dir='AFE\\test'
target_train_dir='FER2013\\train'
target_test_dir='FER2013\\test'

---

## Dataset preparation

In [ ]:
# Define the transformations for resizing, converting to grayscale, normalization

transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])


# Create datasets
source_train_data = datasets.ImageFolder(source_train_dir, transform=transform)
source_test_data = datasets.ImageFolder(source_test_dir, transform=transform)
target_train_data = datasets.ImageFolder(target_train_dir, transform=transform)
target_test_data = datasets.ImageFolder(target_test_dir, transform=transform)
print("imagefolder done")


# Create data loaders for batching the data
source_train_loader = DataLoader(source_train_data, batch_size=batch_size, shuffle=True,num_workers=8)
source_test_loader = DataLoader(source_test_data, batch_size=batch_size, shuffle=True,num_workers=8)
target_train_loader = DataLoader(target_train_data, batch_size=batch_size, shuffle=True,num_workers=8)
target_test_loader = DataLoader(target_test_data, batch_size=batch_size, shuffle=True,num_workers=8)
print("DataLoader done")

In [ ]:
# Checking if the data is made up of tensors for safety
print(f'Type of first instance in source train data: {type(source_train_data[0][0])}')
print(f'Type of first instance in target train data: {type(target_train_data[0][0])}')

---

## Network architecture


## Mobilenetv3

In [ ]:
model=models.mobilenet_v3_small(weights='DEFAULT')
model = model.to(device)


# We're not freezing anything
#for param in model.parameters():
  #param.requires_grad = False


#Changing number of classes of output
print(model.classifier)
model.classifier[3] = nn.Linear(1024, 7).to(device)
print(model.classifier)


# Optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
print(model.classifier)


## Efficientnetv2

In [ ]:
model=models.efficientnet_v2_s(weights='DEFAULT')


model = model.to(device)

#Changing number of classes of output
print(model.classifier)
model.classifier[1] = nn.Linear(1280, 7).to(device)
print(model.classifier)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


---
---
---

# Coral

## Coral Loss


In [ ]:
# CORAL loss
def coral_loss(source, target):
    source_mean = torch.mean(source, dim=0, keepdim=True)
    target_mean = torch.mean(target, dim=0, keepdim=True)
    source_cov = (source - source_mean).t() @ (source - source_mean) / (source.size(0) - 1)
    target_cov = (target - target_mean).t() @ (target - target_mean) / (target.size(0) - 1)
    loss = torch.norm(source_cov - target_cov, p='fro')
    return loss

# Define the task-specific loss
cross_entropy_loss  = nn.CrossEntropyLoss()

In [ ]:
model.train()
total_loss = 0
correct = 0
total = 0

progress_bar = tqdm(enumerate(zip(source_train_loader, target_train_loader)), total=len(source_train_loader))
for epoch in range(num_epochs):
    progress_bar.reset()

    for i, ((source_inputs, source_labels), (target_inputs, _)) in enumerate(zip(source_train_loader, target_train_loader)):

        source_inputs = source_inputs.to(device)
        source_labels = source_labels.to(device)
        target_inputs = target_inputs.to(device)
        
        source_outputs = model(source_inputs)
        target_outputs = model(target_inputs)

        # loss
        cross_entropy_loss_value = cross_entropy_loss(source_outputs, source_labels)
        coral_loss_value  = coral_loss(source_outputs, target_outputs)
        loss = (coral_loss_value*0.7) + (cross_entropy_loss_value*0.3)


        # optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

        # accuracy
        _, predicted = torch.max(source_outputs.data, 1)
        total += source_labels.size(0)
        correct += (predicted == source_labels).sum().item()

        total_loss += loss.item()

        progress_bar.set_description(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / (i + 1)}, Accuracy: {correct / total * 100}%')
        progress_bar.update(1)

    progress_bar.set_description(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(source_train_loader)}, Accuracy: {correct / total * 100}%')
    progress_bar.refresh()

progress_bar.close()

In [ ]:
# Evaluation
model.eval()  # set the model to evaluation mode

correct = 0
total = 0

with torch.no_grad():
    for i, (inputs, labels) in enumerate(target_test_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)

        # Compute accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Test Target Accuracy: {correct / total * 100}%')

correct = 0
total = 0

with torch.no_grad(): #reducing memory consumption
    for i, (inputs, labels) in enumerate(source_test_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)

        # Compute accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'source Test Accuracy: {correct / total * 100}%')

correct = 0
total = 0

with torch.no_grad():
    for i, (inputs, labels) in enumerate(target_train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)

        # Compute accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Target train Accuracy: {correct / total * 100}%')

---

# MMD


In [ ]:
model=models.mobilenet_v3_small(weights='DEFAULT')
model = model.to(device)


# We're not freezing anything
#for param in model.parameters():
  #param.requires_grad = False


#Changing number of classes of output
print(model.classifier)
model.classifier[3] = nn.Linear(1024, 7).to(device)
print(model.classifier)


# Optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
print(model.classifier)

In [ ]:
def gaussian_kernel_matrix(x, y, sigma):
    x_size = x.size(0)
    y_size = y.size(0)
    dim = x.size(1)
    x = x.unsqueeze(1)
    y = y.unsqueeze(0)
    tiled_x = x.expand(x_size, y_size, dim)
    tiled_y = y.expand(x_size, y_size, dim)
    return torch.exp(-((tiled_x - tiled_y)**2).sum(2) / (2.0 * sigma**2))

# gaussian kernel for mmd loss, sigma=1
def compute_mmd(x, y, sigma):
    x_kernel = gaussian_kernel_matrix(x, x, sigma)
    y_kernel = gaussian_kernel_matrix(y, y, sigma)
    xy_kernel = gaussian_kernel_matrix(x, y, sigma)
    mmd = x_kernel.mean() + y_kernel.mean() - 2*xy_kernel.mean()
    return mmd

cross_entropy_loss  = nn.CrossEntropyLoss()

In [ ]:
model.train()
total_loss = 0
correct = 0
total = 0

progress_bar = tqdm(enumerate(zip(source_train_loader, target_train_loader)), total=len(source_train_loader))
for epoch in range(num_epochs):
    progress_bar.reset()

    for i, ((source_inputs, source_labels), (target_inputs, _)) in enumerate(zip(source_train_loader, target_train_loader)):
        source_inputs = source_inputs.to(device)
        source_labels = source_labels.to(device)
        target_inputs = target_inputs.to(device)
        source_outputs = model(source_inputs)
        target_outputs = model(target_inputs)

        # loss
        cross_entropy_loss_value = cross_entropy_loss(source_outputs, source_labels)
        mmd_loss_value  = compute_mmd(source_outputs, target_outputs,1)
        loss = (mmd_loss_value*0.5) + (cross_entropy_loss_value*0.5)


        # optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

        # accuracy
        _, predicted = torch.max(source_outputs.data, 1)
        total += source_labels.size(0)
        correct += (predicted == source_labels).sum().item()

        total_loss += loss.item()

        progress_bar.set_description(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / (i + 1)}, Accuracy: {correct / total * 100}%')
        progress_bar.update(1)

    progress_bar.set_description(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(source_train_loader)}, Accuracy: {correct / total * 100}%')
    progress_bar.refresh()

progress_bar.close()

In [ ]:
# Evaluation
model.eval() 
correct = 0
total = 0

with torch.no_grad():
    for i, (inputs, labels) in enumerate(target_test_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)

        # accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Test Target Accuracy: {correct / total * 100}%')

correct = 0
total = 0

with torch.no_grad(): 
    for i, (inputs, labels) in enumerate(source_test_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)

        # Compute accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Source Test Accuracy: {correct / total * 100}%')

correct = 0
total = 0

with torch.no_grad(): 
    for i, (inputs, labels) in enumerate(target_train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)

        # Compute accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Target train Accuracy: {correct / total * 100}%')

---

# Oracle Predictor

In [ ]:
model=models.mobilenet_v3_small(weights='DEFAULT')
model = model.to(device)


# We're not freezing anything
#for param in model.parameters():
  #param.requires_grad = False


#Changing number of classes of output
print(model.classifier)
model.classifier[3] = nn.Linear(1024, 7).to(device)
print(model.classifier)


# Optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
print(model.classifier)


In [ ]:
cross_entropy_loss = nn.CrossEntropyLoss()
model.train()
progress_bar = tqdm(total=len(target_train_loader))
for epoch in range(num_epochs):
    
    total_loss = 0
    correct = 0
    total = 0
    progress_bar.reset()

    for i, (source_inputs, source_labels) in enumerate(target_train_loader):

        source_inputs = source_inputs.to(device)
        source_labels = source_labels.to(device)
        source_outputs = model(source_inputs)

        loss = cross_entropy_loss(source_outputs, source_labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # accuracy
        _, predicted = torch.max(source_outputs.data, 1)
        total += source_labels.size(0)
        correct += (predicted == source_labels).sum().item()

        total_loss += loss.item()

        progress_bar.set_description(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / (i + 1)}, Accuracy: {correct / total * 100}%')
        progress_bar.update(1)

    progress_bar.set_description(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(target_train_loader)}, Accuracy: {correct / total * 100}%')
    progress_bar.refresh()

progress_bar.close()


In [ ]:
model.eval() 
correct = 0
total = 0

with torch.no_grad():
    for i, (inputs, labels) in enumerate(target_test_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)

        # Compute accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Test Target Accuracy: {correct / total * 100}%')
